In [2]:
"""
Denne koden henter laksedata fra SSB, simulerer manglende verdier (NaN),
og viser hvordan disse kan identifiseres og rettes opp igjen ved hjelp av Pandas.
Brukes til å demonstrere datarensing og gjenoppretting av manglende verdier i prosjektet.
"""

import requests
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

# URL til SSB API for tabell 03024
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""

    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")

def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""

    dimension_names = list(data["dimension"].keys())
    values = data["value"]

    # Dette viser kombinasjoner av dimensjoner
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    # Lager DataFrame og knytter verdiene 
    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    # Deler opp dataen i et pivot diagram 
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]


    # Deler opp "Tid" i År og Uke for å gjøre det lettere å analysere 
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
    df_pivot["Uke"] = df_pivot["Tid"].str[5:].astype(int)
   

    # Fjerner "Tid" og omorganiser kolonner slik at År og Uke kommer som den første kolonnen
    df_pivot = df_pivot.drop(columns=["Tid"])
    kolonnerekkefølge = ["År", "Uke"] + [col for col in df_pivot.columns if col not in ["År", "Uke"]]
    df_pivot = df_pivot[kolonnerekkefølge]

    # Lager en uke-dato-kolonne for plotting (mandag i hver uke)
    df_pivot["UkeDato"] = pd.to_datetime(df_pivot["År"].astype(str) + "-" + df_pivot["Uke"].astype(str).str.zfill(2) + "-1", format="%Y-%W-%w")


    return df_pivot

def main():
    """Hovedfunksjon for å hente og prosessere data."""
    try:
        data = fetch_data()
        df_pivot = process_data(data)
        print(df_pivot.to_string(index=False))
        print("Data hentet, sortert og formatert for visning.")
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()

HTTPSConnectionPool(host='data.ssb.no', port=443): Max retries exceeded with url: /api/v0/no/table/03024/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10f7befd0>: Failed to resolve 'data.ssb.no' ([Errno 8] nodename nor servname provided, or not known)"))


In [3]:
"""
Koden henter laksedata fra SSB og gjør dataen om til en tabell for alle verdiene som brukes videre i prosjektet ved hjelp av Pandas.
"""

import requests
import pandas as pd
from itertools import product

# URL til SSB API for tabell 03024
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")

def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
   

    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))
    

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values  
    

    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    
    
    df_pivot.columns = ["År og ukenr.", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    
    return df_pivot

def main():
    """Hovedfunksjon for å hente og prosessere data."""
    try:
        data = fetch_data()
        df_pivot = process_data(data)
        print(df_pivot.to_string(index=False))
        print("Data hentet, sortert og formatert for visning.")
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()

HTTPSConnectionPool(host='data.ssb.no', port=443): Max retries exceeded with url: /api/v0/no/table/03024/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10f7bfb10>: Failed to resolve 'data.ssb.no' ([Errno 8] nodename nor servname provided, or not known)"))
